In [88]:
import numpy as np

import pandas as pd
from pandas.api.types import CategoricalDtype

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import zipfile
import os

from ds100_utils import run_linear_regression_test

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline

# Plot settings
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

In [13]:
with zipfile.ZipFile('cook_county_data.zip') as item:
    item.extractall()

In [14]:
training_data = pd.read_csv("cook_county_train.csv", index_col='Unnamed: 0')

In [58]:
##downloaded data from Mapping Inequality Redlining in New Deal America, converted shapefile to CSV

with zipfile.ZipFile('mygeodata.zip') as item:
    item.extractall()

In [59]:
Holc_grade = pd.read_csv("cartodb-query.csv")

In [71]:
##Rename X and Y to Long, Lat and Dropped last 4 digits of Longitude and 

Holc_grade = Holc_grade.rename(columns={"X": "Longitude", "Y": "Latitude"})

Holc_grade[["Latitude", "Longitude"]] = Holc_grade[["Latitude", "Longitude"]].round(3)

In [87]:
training_data[["Latitude", "Longitude"]] = training_data[["Latitude", "Longitude"]].round(3)
training_data

,PIN,Property Class,Neighborhood Code,Land Square Feet,Town Code,Apartments,Wall Material,Roof Material,Basement,Basement Finish,...,Sale Month of Year,Sale Half of Year,Most Recent Sale,Age Decade,Pure Market Filter,Garage Indicator,Neigborhood Code (mapping),Town and Neighborhood,Description,Lot Size
0,17294100610000,203,50,2500.0,76,0.0,2.0,1.0,1.0,3.0,...,9,2,1.0,13.2,0,0.0,50,7650,"This property, sold on 09/14/2015, is a one-st...",2500.0
1,13272240180000,202,120,3780.0,71,0.0,2.0,1.0,1.0,1.0,...,5,1,1.0,9.6,1,1.0,120,71120,"This property, sold on 05/23/2018, is a one-st...",3780.0
2,25221150230000,202,210,4375.0,70,0.0,2.0,1.0,2.0,3.0,...,2,1,0.0,11.2,1,1.0,210,70210,"This property, sold on 02/18/2016, is a one-st...",4375.0
3,10251130030000,203,220,4375.0,17,0.0,3.0,1.0,1.0,3.0,...,7,2,1.0,6.3,1,1.0,220,17220,"This property, sold on 07/23/2013, is a one-st...",4375.0
4,31361040550000,202,120,8400.0,32,0.0,3.0,1.0,2.0,3.0,...,6,1,0.0,6.3,1,1.0,120,32120,"This property, sold on 06/10/2016, is a one-st...",8400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204787,25163010260000,202,321,4375.0,72,0.0,2.0,1.0,1.0,1.0,...,7,2,0.0,5.8,1,1.0,321,72321,"This property, sold on 07/23/2014, is a one-st...",4375.0
204788,5063010090000,204,21,16509.0,23,0.0,1.0,1.0,1.0,1.0,...,3,1,1.0,9.3,1,1.0,21,2321,"This property, sold on 03/27/2019, is a one-st...",16509.0
204789,16333020150000,202,90,3810.0,15,0.0,2.0,1.0,1.0,1.0,...,1,1,1.0,5.9,1,1.0,90,1590,"This property, sold on 01/31/2014, is a one-st...",3810.0
204790,9242030500000,203,80,6650.0,22,0.0,2.0,1.0,1.0,3.0,...,2,1,1.0,6.0,1,1.0,80,2280,"This property, sold on 02/22/2018, is a one-st...",6650.0


In [84]:
with_grade = training_data.merge(Holc_grade, on = ["Latitude", "Longitude"])

In [89]:
with_grade

,PIN,Property Class,Neighborhood Code,Land Square Feet,Town Code,Apartments,Wall Material,Roof Material,Basement,Basement Finish,...,Age Decade,Pure Market Filter,Garage Indicator,Neigborhood Code (mapping),Town and Neighborhood,Description,Lot Size,name,holc_id,holc_grade
0,10222050360000,205,81,4913.0,24,0.0,3.0,1.0,1.0,3.0,...,6.6,1,1.0,81,2481,"This property, sold on 12/21/2016, is a two-st...",4913.0,NaN,B32,B
1,10222050510000,203,81,4920.0,24,0.0,2.0,1.0,1.0,1.0,...,6.0,1,1.0,81,2481,"This property, sold on 07/06/2017, is a one-st...",4920.0,NaN,B32,B
2,15152220190000,203,31,5280.0,31,0.0,2.0,1.0,1.0,3.0,...,9.4,0,1.0,31,3131,"This property, sold on 06/21/2019, is a one-st...",5280.0,NaN,B87,B
3,15152220070000,203,31,9939.0,31,0.0,2.0,1.0,1.0,3.0,...,9.6,1,1.0,31,3131,"This property, sold on 05/08/2017, is a one-st...",9939.0,NaN,B87,B
4,15152220120000,203,31,6860.0,31,0.0,4.0,1.0,1.0,3.0,...,9.4,0,1.0,31,3131,"This property, sold on 09/13/2019, is a one-st...",6860.0,NaN,B87,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011,5161060230000,206,170,15000.0,23,0.0,4.0,3.0,3.0,1.0,...,9.0,1,1.0,170,23170,"This property, sold on 04/16/2013, is a two-st...",15000.0,NaN,A16,A
1012,20293210250000,203,221,5400.0,72,0.0,2.0,1.0,1.0,3.0,...,10.0,1,1.0,221,72221,"This property, sold on 01/12/2015, is a one-st...",5400.0,NaN,C227,C
1013,15353100170000,202,70,3050.0,34,0.0,2.0,1.0,1.0,3.0,...,8.6,1,1.0,70,3470,"This property, sold on 07/03/2017, is a one-st...",3050.0,NaN,C182,C
1014,10074010430000,205,23,11475.0,24,0.0,3.0,1.0,1.0,1.0,...,6.5,1,1.0,23,2423,"This property, sold on 06/03/2016, is a two-st...",11475.0,NaN,B29,B


In [ ]:
#decision tree

from sklearn import tree
decision_tree_model = tree.DecisionTreeClassifier()
decision_tree_model = decision_tree_model.fit(with_grade[["Neighborhood Code", "Lot Size"]], with_grade["holc_grade"])

In [ ]:
decision_tree_model

In [ ]:
tree.plot_tree(decision_tree_model)

In [ ]:
import graphviz

dot_data = tree.export_graphviz(decision_tree_model, out_file=None, 
                      feature_names=["Neighborhood Code", "Lot Size"],  
                      class_names=["A", "B", "C", "D"],  
                      filled=True, rounded=True,  
                      special_characters=True)  
graph = graphviz.Source(dot_data)
#graph.render(format="png", filename="iris_tree")
graph